<a href="https://colab.research.google.com/github/MasaoMori/Tokyo-Ward-Assembly-Election/blob/main/untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 準備
import pandas as pd

pd.options.display.float_format = '{:,.0f}'.format

ele_dat = pd.read_excel('./区議会議員選挙過去データ.xlsx', 
                        sheet_name="選挙結果",
                        converters={'有権者数':int})
ele_par = pd.read_excel('./区議会議員選挙過去データ.xlsx', sheet_name="政党名")

ele_result = pd.merge(ele_dat, ele_par, on="党派")

データはこんな感じで出ています。１レコード、1人の立候補者のデータです。末尾のカラムには、選挙区での概要情報がつけてあります。

In [2]:
ele_result

,年,区名,当落,得票数,氏名,よみ,年齢,性別,党派,新旧,...,投票日,告知日,定数,候補者数,有権者数,投票率%,投票者数,得票率%,略称党名,党順序
0,2011,目黒,当,"3,986",須藤 甚一郎,すどう じんいちろう,72,男,無所属,現,...,2011-04-24,2011-04-17,35,55,213142,39,"83,040",5,無所属,11
1,2011,目黒,当,"1,845",梅原 辰郎,うめはら たつろう,63,男,無所属,現,...,2011-04-24,2011-04-17,35,55,213142,39,"83,040",2,無所属,11
2,2011,目黒,当,"1,516",坂本 史子,さかもと ふみこ,55,女,無所属,現,...,2011-04-24,2011-04-17,35,55,213142,39,"83,040",2,無所属,11
3,2011,目黒,落,855,伊東 たつみ,いとう たつみ,34,女,無所属,新,...,2011-04-24,2011-04-17,35,55,213142,39,"83,040",1,無所属,11
4,2011,目黒,落,733,高林 邦年,たかばやし くにとし,34,男,無所属,新,...,2011-04-24,2011-04-17,35,55,213142,39,"83,040",1,無所属,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,2019,中央,落,458,林 敦,NaN,52,男,あたらしい党,新,...,2019-04-21,2019-04-14,30,44,128000,46,"58,419",1,諸派,99
923,2019,渋谷,当,"1,631",堀切 稔仁,NaN,50,男,渋谷オンブスマン,現,...,2019-04-21,2019-04-14,34,55,184602,40,"73,767",2,諸派,99
924,2019,渋谷,落,288,新庄 美鈴,NaN,58,女,自由を守る会,新,...,2019-04-21,2019-04-14,34,55,184602,40,"73,767",0,諸派,99
925,2019,台東,当,"2,564",中村 謙治郎,なかむら けんじろう,41,男,都ファ,新,...,2019-03-17,2015-03-10,32,41,160295,43,"68,238",4,諸派,99


###　1. 過去3回の区議会選挙における、有権者数と投票者数の推移

In [3]:
voters_tmp = ele_result[['年','区名','有権者数','投票者数']]
# 有権者数と投票者数を年、区別に抽出

voters_q = voters_tmp[~(voters_tmp.duplicated())]
# 当選落選者の選挙結果のデータにしているので重複しています、重複を削除。

voters = voters_q.pivot_table(index=['区名','年','有権者数','投票者数'],
                              aggfunc=sum)
                            # 区、年、で有権者数、投票者数を算出 
voters # 結果表示

Empty DataFrame
Columns: []
Index: [(中央, 2011, 97887, 44626.683300000004), (中央, 2015, 110176, 50284.3264), (中央, 2019, 128000, 58419.2), (台東, 2011, 143834, 71298.5138), (台東, 2015, 151090, 66706.235), (台東, 2019, 160295, 68237.5815), (品川, 2011, 296585, 120354.19299999998), (品川, 2015, 302599, 121281.6792), (品川, 2019, 322379, 128048.93879999999), (大田, 2011, 561414, 243036.1206), (大田, 2015, 572311, 242316.47740000003), (大田, 2019, 599370, 256170.738), (渋谷, 2011, 168344, 67707.9568), (渋谷, 2015, 176703, 73102.0311), (渋谷, 2019, 184602, 73766.9592), (目黒, 2011, 213142, 83040.1232), (目黒, 2015, 219196, 86253.62599999999), (目黒, 2019, 228386, 91993.8808)]

### 2. 過去3回の区議会選挙における、主要政党の得票数

In [33]:
#party_name = ele_par[~(ele_par['略称党名'] == '諸派')][['略称党名','党順序']]
# 諸派を除く政党の実績を上記データから抽出

import numpy as np # 計算ライブラリを読み込む

pv = ele_result.pivot_table(index=['略称党名','党順序','年'], 
                            columns='区名', 
                            aggfunc=sum, 
                            values=['得票数'])
# ピボット集計する。政党、年で各政党がどれくらい得票したか

v = pv.loc[['自民','民主','立憲','国民','共産','社民','公明','維新','NHK'],:].applymap('{:,.0f}'.format)
v.reset_index(level='党順序',drop=True)
#　データを整形、実績ある政党の数値を抽出


得票数                                        
区名             中央      台東      品川      大田      渋谷      目黒
略称党名 年                                                   
自民   2011  15,996  19,530  31,827  69,680  17,717  25,556
     2015  16,881  21,653  31,827  74,119  19,725  27,897
     2019  21,447  20,180  31,987  72,330  17,441  26,956
民主   2011   1,725  14,696  15,060  38,732  11,350  14,094
     2015   1,143   9,914  15,060  22,724   9,329   8,978
立憲   2019   4,278   3,679   7,846  15,796   9,755  13,295
国民   2019     485   4,663   6,404   3,935     603     nan
共産   2011   3,970   7,323  17,302  31,713   8,632   8,403
     2015   5,125   8,086  17,302  37,031  10,616  11,685
     2019   4,128   7,564  19,228  30,373   8,004   9,323
社民   2011     nan     418   1,115   1,527     nan   1,050
     2015     nan     652   1,115     nan     nan     950
     2019     nan     nan     nan     nan     nan     572
公明   2011   6,079  10,099  20,031  45,924   9,172  10,669
     2015   5,263   9,313  20,031  41,479   8,500  10,163
     2019   5,702   9,580  18,634  46,702   7,855  11,226
維新   2015   3,080   1,882     nan   8,893   2,074   3,917
     2019   1,108     nan   2,844   5,448     nan   2,048
NHK  2019   1,100   1,656   2,532   3,036   1,428   1,979

### 3. 過去3回の区議会選挙における、当選者のみ主要政党の得票数

In [34]:
import numpy as np # 計算ライブラリを読み込む

qv = ele_result[ele_result['当落']=='当'].pivot_table(index=['略称党名','党順序','年'], 
                                                   columns='区名',
                                                   aggfunc=sum,
                                                   values=['得票数'])
# ピボット集計する。当選議員のみ、政党、年で各政党がどれくらい得票したか

qv = qv.loc[['自民','民主','立憲','国民','共産','公明','維新','NHK'],:].applymap('{:,.0f}'.format)
qv.reset_index(level='党順序',drop=True)
#　データを整形、実績ある政党の数値を抽出


得票数                                        
区名             中央      台東      品川      大田      渋谷      目黒
略称党名 年                                                   
自民   2011  14,572  17,171  28,621  67,792  13,690  24,317
     2015  16,104  19,908  28,621  69,658  17,724  23,121
     2019  19,536  14,895  30,143  62,707  14,417  19,877
民主   2011   1,725  12,420  11,180  22,505   9,702   8,360
     2015   1,143   9,914  11,180  18,817   8,339   6,537
立憲   2019   4,278   3,679   6,069  15,796   9,755  13,295
国民   2019     nan   4,663   1,994     nan     nan     nan
共産   2011   3,970   7,323  15,633  31,713   8,632   7,142
     2015   5,125   8,086  15,633  37,031  10,616  11,685
     2019   2,396   7,564  16,280  28,146   5,984   9,323
公明   2011   6,079  10,099  20,031  45,924   9,172  10,669
     2015   5,263   9,313  20,031  41,479   8,500  10,163
     2019   5,702   9,580  18,634  46,702   7,855  11,226
維新   2015   3,080   1,167     nan   8,893     nan   3,917
     2019   1,108     nan   2,844   5,448     nan   2,048
NHK  2019   1,100   1,656   2,532   3,036   1,428   1,979

### 4. 全得票、当選議員のみ得票でどれくらい差があるか？

In [39]:
#現在絶賛処理中

pvv = pv
pvv=pvv.reset_index(drop=False)
pvv.columns = pvv.columns.droplevel(0)

qvv = qv
qvv=qvv.reset_index(drop=False)
qvv.columns = qvv.columns.droplevel(0)